In [ ]:
import sys
import os
import numpy as np
import time
import pickle as pk
import pandas as pd
import logging
logging.getLogger("qiskit").setLevel(logging.WARNING)
logging.getLogger("stevedore").setLevel(logging.WARNING)

from loguru import logger
logger.remove()

sys.path.insert(0, "../")
# sys.path.insert(0, "code/")

from generate_qc import compute_result_from_qc

# num_var, num_sol, seed, sols, eqs
# eqs_df = pk.load(open("output/eqs.pkl", "rb"))  # type: pd.DataFrame
eqs_df = pd.read_csv("output/eqs.csv", index_col=0)
eqs_df.eqs = eqs_df.eqs.apply(lambda x: [np.array(i) for i in eval(x)])
eqs_df.sol = eqs_df.sol.apply(lambda x: eval(x))

eqs_df = eqs_df[eqs_df.num_var.isin([15, 20, 25]) & eqs_df.num_sol.isin([1])]


In [ ]:
eqs_df

In [ ]:
df15 = eqs_df[eqs_df.num_var == 15][:15]
df20 = eqs_df[eqs_df.num_var == 20][:15]
df25 = eqs_df[eqs_df.num_var == 25][:15]


In [ ]:

num_vars = {
    15: df15,
    20: df20,
    25: df25
}
specs = {
    15: [[7, 1], [4, 2], [5, 2]],
    20: [[4, 2], [5, 2], [5, 3]],
    25: [[5, 2], [6, 2], [5, 3]]
}
for n, df in num_vars.items():
    for use_ancilla, level in specs[n]:
        without_rearrange_res = []
        with_rearrange_res = []
        for i, row in df.iterrows():
            eqs = row["eqs"]
            num_var = row["num_var"]
            num_sol = row["num_sol"]
            seed = row["seed"]
            sol = row["sol"]

            res = compute_result_from_qc(
                num_var, seed, shots=1024,
                split=-1, level=level, num_sols=num_sol,
                use_ancilla=use_ancilla, iterations=3,
                solutions=sol, eqs=eqs, dry_run=True,
                arrange=False
            )
            depth = (res["depth of qc"]-2) / res["num of iterating G"]
            # print(f"without arrange: m={use_ancilla}, level={level}, avg depth={depth}")
            without_rearrange_res.append(depth)
            res = compute_result_from_qc(
                num_var, seed, shots=1024,
                split=-1, level=level, num_sols=num_sol,
                use_ancilla=use_ancilla, iterations=3,
                solutions=sol, eqs=eqs, dry_run=True,
                arrange=True
            )
            depth = (res["depth of qc"]-2) / res["num of iterating G"]
            # print(f"with arrange: m={use_ancilla}, level={level}, avg depth={depth}")
            with_rearrange_res.append(depth)
        avg1 = sum(without_rearrange_res) / len(without_rearrange_res)
        avg2 = sum(with_rearrange_res) / len(with_rearrange_res)
        print(f"n={n}, m={use_ancilla}, level={level}, 15eq avg without rearrange: {avg1}, with rearrange: {avg2}")
